initialize_NLP() was modified from dev version to allow for kwargs...

In [2]:
using NLOptControl
using Plots
using Polynomials
using FastGaussQuadrature
using Parameters

In [19]:
# initialize basic problem definition
ps, nlp = initialize_NLP(numStates=1,numControls=1,Ni=2,Nck=[3, 3]);

####################################
# perform analytical calcualtions -> for plots
####################################
@unpack t0, tf = ps
t0 = Float64(0); tf = Float64(10);
@pack ps = t0, tf
t = Array(linspace(t0,tf,100));
α₁ =  -0.3; α₂ = 3; α₃ = -8; α₄ =  7;

γ = Poly([α₁,α₂,α₁]); #TODO check on that imported binding warning
y = polyval(γ,t);

# evaluate the integral
∫γ = polyint(γ);
Y = polyval(∫γ,t[end]) - polyval(∫γ,t[1]);
C = Y - polyval(∫γ,t[end]); # constant of integration
∫y = polyval(∫γ,t) + C;

# evaluate the derivative
dγ = polyder(γ);
dy = polyval(dγ,t);

##TEMP## -->to get fake optimization data
@unpack Nck, Ni, t0, tf = ps
taus_and_weights = [gaussradau(Nck[int]) for int in 1:Ni];
τ = [taus_and_weights[int][1] for int in 1:Ni];
ω = [taus_and_weights[int][2] for int in 1:Ni];
di, tm, t_data, ωₛ=create_intervals(t0,tf,Ni,Nck,τ,ω);
@pack ps = τ, ω


@unpack decisionVector, lengthControlVector = nlp
fake_control_data = zeros(lengthControlVector,);
ts = [t_data[1]; t_data[2]]
if Ni > 2 
    error("fix ts")
end
fake_state_data = polyval(γ,ts);
decisionVector=[fake_state_data[:];fake_control_data;t0;tf]; # for now looking at no controls
@pack  nlp = decisionVector

##TEMP## -->to get fake optimization data

16-element Array{Float64,1}:
 -0.3    
  4.08031
  7.01969
  7.2    
  7.2    
  6.25454
  1.84546
 -0.3    
  0.0    
  0.0    
  0.0    
  0.0    
  0.0    
  0.0    
  0.0    
 10.0    

In [23]:
ps

NLOptControl.PS_data
  Nck: [3,3]
  Ni: 2
  τ: Array{Float64,1}[[-1.0,-0.289898,0.689898],[-1.0,-0.289898,0.689898]]
  ω: Array{Float64,1}[[0.222222,1.02497,0.752806],[0.222222,1.02497,0.752806]]
  t0: 0.0
  tf: 10.0
  stateMatrix: Array{Float64,2}[
[0.0; 0.0; 0.0; 0.0],

[0.0; 0.0; 0.0; 0.0]]
  controlMatrix: Array{Float64,2}[
[0.0; 0.0; 0.0],

[0.0; 0.0; 0.0]]

In [24]:
nlp

NLOptControl.NLP_data
  numStates: 1
  numStatePoints: [4,4]
  lengthStateVector: 8
  numControls: 1
  numControlPoints: [3,3]
  lengthControlVector: 6
  lengthDecVector: 16
  stateStartIdx: 1
  stateStopIdx: 8
  controlStartIdx: 9
  controlStopIdx: 14
  timeStartIdx: 15
  timeStopIdx: 16
  stateVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  controlVector: [0.0,0.0,0.0,0.0,0.0,0.0]
  decisionVector: [-0.3,4.08031,7.01969,7.2,7.2,6.25454,1.84546,-0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0]

In [22]:
function nlp2ocp(decisionVector::Array{Float64,1},nlp::NLP_data,ps::PS_data)
    @unpack t0, tf, stateMatrix, controlMatrix, Ni = ps
    @unpack stateVector,stateStartIdx,stateStopIdx,numStates,numStatePoints = nlp 
    @unpack controlVector,controlStartIdx,controlStopIdx,numControls,numControlPoints = nlp 
    @unpack timeStartIdx,timeStopIdx = nlp 
    # @unpack_* macro will shadow input argument with the same name as a type-fieldname; see--> http://parametersjl.readthedocs.io/en/latest/manual/#parameters-manual
    # this can be a HUGE headache!!
    
    # update parameters 
    stateVector = decisionVector[stateStartIdx:stateStopIdx]; 
    controlVector = decisionVector[controlStartIdx:controlStopIdx];
    t0 = decisionVector[timeStartIdx];
    tf = decisionVector[timeStopIdx];
    
    # break up problem to evaluate constraints
    for state in 1:numStates
        for int in 1:Ni # intervals
            if int == 1
                stateMatrix[:,1] = stateVector[state*numStatePoints:(state+1)*numStatePoints];
            else
                stateMatrix[:,int] = stateVector[state*int*numStatePoints:(state*int+1)*numStatePoints];
            end
        end
    end
    for control in 1:numControls
        for int in 1:Ni # intervals
            if int == 1 
                controlMatrix[:,1] = controlVector[control:control*numControlPoints];
            else
                controlMatrix[:,int] = controlVector[control*int*numControlPoints:(control*int+1)*numControlPoints];
            end
        end
    end

    @pack ps = t0, tf, stateMatrix, controlMatrix 
    @pack nlp = stateVector, controlVector # TODO may not be necessary to update this or even have this...they could be local variables
end

nlp2ocp(decisionVector,nlp,ps);

@unpack_PS_data ps
@unpack_NLP_data nlp
print(nlp,"\n","\n")
print(ps)

LoadError: MethodError: no method matching one(::Array{Int64,1})
Closest candidates are:
  one(!Matched::Type{Measures.Length{:mm,Float64}}) at /home/febbo/.julia/v0.5/Plots/src/layouts.jl:17
  one(!Matched::Type{Measures.Length{:pct,Float64}}) at /home/febbo/.julia/v0.5/Plots/src/layouts.jl:35
  one(!Matched::BitArray{2}) at bitarray.jl:413
  ...

In [ ]:
 A=[4,4,4]

In [ ]:
idx = cumsum(A)

In [ ]:
# defect constraints - > the states meet at the end and beginning of each mesh interval (2,3..,Ni)
xd=lgr_diff(Nc,Ni,x,t_data)

In [ ]:
Ni